## Dupla inga szimulációja szimbolikus differenciálással

Szokásos Lagrange-függvényes módszer, de a mozgásegyenleteket most nem kézzel számoljuk, hanem a `Sympy` csomaggal.

In [1]:
using DifferentialEquations
using SymPy
using PyCall
using Plots
PyCall.pyimport_conda("sympy.physics.mechanics", "sympy");

In [2]:
l1,l2,m1,m2,g,t = @syms l1 l2 m1 m2 g t
q  = [f(t) for f in @symfuns θ1 θ2]
qd = [f.diff(t) for f in q];

In [3]:
θ1,θ2   = q
θ1d,θ2d = qd

x1,y1 = l1*sin(θ1),-l1*cos(θ1)
x2,y2 = x1+l2*sin(θ2),y1-l2*cos(θ2);

In [4]:
v1,v2 = x1.diff(t)^2 + y1.diff(t)^2, x2.diff(t)^2 + y2.diff(t)^2
Lag   = (m1*v1/2 + m2*v2/2 - m1*g*y1 - m2*g*y2).simplify()
LM    = sympy.physics.mechanics.LagrangesMethod(Lag,[θ1,θ2])
LM.form_lagranges_equations();

In [5]:
u  = vcat(qd,q)
M_ = LM.mass_matrix.subs(zip(u,["u$i" for i in 1:length(u)]))
f_ = LM.forcing.subs(zip(u,["u$i" for i in 1:length(u)]))
eval(Meta.parse("M(u,t) = " * reduce(replace,[r"u(\d+)"=>s"u[\1]","Sym"=>""],init=string(M_))))
eval(Meta.parse("f(u,t) = " * reduce(replace,[r"u(\d+)"=>s"u[\1]","Sym"=>""],init=string(f_))));

In [6]:
function pendulum!(du,u,p,t)
    du[1:2] = M(u,t)\f(u,t) .- α.*u[1:2]
    du[3:4] = u[1:2]
end;

In [7]:
l1,l2,m1,m2,g = .75,.25,1.,1.,9.81
α    = [0.05,0.05]
u0   = [0.,0.,pi/2,0.]
prob = ODEProblem(pendulum!,u0,(0.,15.))
sol  = DifferentialEquations.solve(prob,Tsit5(),adaptive=false,dt=0.001);

In [8]:
anim = @animate for t in 1:length(sol)
    θ1,θ2 = sol.u[t][3:4]
    x1,y1 = l1*sin(θ1),-l1*cos(θ1)
    x2,y2 = x1+l2*sin(θ2),y1-l2*cos(θ2)
    
    plot([0,x1,x2],[0,y1,y2],xlims=(-1.2,1.2),ylims=(-1.2,1.2),title=string(t),aspect_ratio=:equal,legend=:none)
    plot!([0,x1,x2],[0,y1,y2],seriestype=:scatter)
end every 50
gif(anim,"pendulum.gif")

┌ Info: Saved animation to 
│   fn = /home/bj0rn/Desktop/szakkör/fiz/pendulum.gif
└ @ Plots /home/bj0rn/.julia/packages/Plots/jWNMG/src/animation.jl:90


Plots.AnimatedGif("/home/bj0rn/Desktop/szakkör/fiz/pendulum.gif")